In [1]:
import numpy as np
import pandas as pd

In [2]:
users = pd.read_csv('../../data/Users.csv', delimiter=';')
books = pd.read_csv('../../data/Books.csv', delimiter=';', dtype={'ISBN': str, 'Title': str, 'Author': str, 'Year': np.int16, 'Publisher': str})
ratings = pd.read_csv('../../data/Ratings.csv', delimiter=';', dtype={'User-ID': np.int32, 'ISBN': str, 'Rating': np.int8})

/var/folders/yn/16crnc792zg2yr78zf0qg0j40000gq/T/ipykernel_44530/2231409931.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('../../data/Users.csv', delimiter=';')


In [3]:
# from scipy.sparse import csr_matrix
# from sklearn.metrics.pairwise import cosine_similarity

# def list_minus(a, b):
#     return [x for x in a if x not in b]

# class ItemRecommender():
#     def __init__(self):
#         super().__init__()
#         self.n_recomm = 5

#     def fit(self, items, users, ratings):
#         books = items.reset_index() # add index as a column
#         isbn_mapping = {category: idx for idx, category in enumerate(books['ISBN'])}
        
#         ratings = ratings.copy()
#         ratings['ISBN_i'] = ratings['ISBN'].map(isbn_mapping) # map ISBN to index
#         ratings.dropna(subset=['ISBN_i'], inplace=True) # drop rows with NaN ISBN_i
#         ratings['ISBN_i'] = ratings['ISBN_i'].astype(np.int32)

#         self.ratings = ratings

#         # Create a sparse user-item matrix
#         user_item_matrix = csr_matrix((ratings['Rating'], (ratings['User-ID'], ratings['ISBN_i'])), dtype=np.float64)

#         # Normalize the user-item matrix
#         normalized_matrix = user_item_matrix.copy()
#         self.means = np.array([normalized_matrix[i].data.mean() for i in range(normalized_matrix.shape[0])])
#         normalized_matrix.data -= np.repeat(self.means, np.diff(normalized_matrix.indptr))
#         self.normalized_matrix = normalized_matrix

#         self.similarity_matrix = cosine_similarity(normalized_matrix.T, dense_output=False)
#         self.books = books
        
#     def predict(self, users, items):
#         user_predictions = {}
#         for user_id in users:
#             # Get the row corresponding to the user
#             user_row = self.normalized_matrix.getrow(user_id)
#             top_rated = np.argsort(user_row.data)[::-1][:self.n_recomm]

#             recommended_books = []
#             for book_i in top_rated:
#                 book_row = self.similarity_matrix.getrow(book_i)
#                 most_similar_books = np.argsort(book_row.data)[::-1][:self.n_recomm]
#                 recommended_books.extend(most_similar_books)

#             # print(f"User recommendations: {top_rated}")
#             # print(f"Recommended books: {recommended_books}")
#             recommended_books = list_minus(set(recommended_books), user_row.indices)
#             # print(f"Recommended books: {recommended_books}")

#             average_ratings = {}
#             for book in recommended_books:
#                 # Get the average rating for the book
#                 average_rating = self.normalized_matrix.getcol(book).data.mean()
#                 average_ratings[book] = average_rating

#             sorted_recommended_books = sorted(recommended_books, key=lambda book: average_ratings[book], reverse=True)[:self.n_recomm]
#             # print(f"Sorted recommended books: {sorted_recommended_books}")
#             isbns = self.ratings[self.ratings['ISBN_i'].isin(sorted_recommended_books)]['ISBN']
#             # print(f"ISBNs: {isbns}")
#             user_predictions[user_id] = self.books[self.books['ISBN'].isin(isbns)]

#         return user_predictions

#     def eval(self, gt_ratings, pred_ratings):
#         pass

In [4]:
import sys
sys.path.append('../..')

from src.evaluation import average_precision
from sklearn.model_selection import train_test_split

In [5]:
train_ratings, test_ratings = train_test_split(ratings, test_size=0.3, random_state=42)

In [6]:
import src.recommenders.item_collaborative_filtering as icf

model = icf.ItemRecommender()
model.fit(users=users, items=books, ratings=train_ratings)

/Users/oleh.sannikov/Documents/Навчання/Науки про дані/RecSys/CP/rs-ucu-books-recommender/experiments/sunnycows/../../src/recommenders/item_collaborative_filtering.py:32: RuntimeWarning: Mean of empty slice.
  self.means = np.array([normalized_matrix[i].data.mean() for i in range(normalized_matrix.shape[0])])
/Users/oleh.sannikov/miniconda3/envs/recsys/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [22]:
test_users = test_ratings['User-ID'].unique()[:1000]
len(test_users)

1000

In [23]:
books.shape

(271379, 5)

In [24]:
best_items = model.predict(test_users, items=books)

In [20]:
len(best_items)

100

In [14]:
#best_items

In [25]:
from tqdm import tqdm

rating_mask = test_ratings['Rating'] >= 5

mean_ap = 0
for i in tqdm(range(len(test_users))):
    user_mask = test_ratings['User-ID'] == test_users[i]
    relevant_items = test_ratings.loc[(user_mask & rating_mask)].sort_values(by='Rating', ascending=False)
    if len(relevant_items) == 0:
        continue
    user_id = test_users[i]
    ap = average_precision(best_items[user_id]['ISBN'].to_numpy(), relevant_items['ISBN'].to_numpy(), k=5)
    # print(f"Relevant items: {relevant_items['ISBN'].to_numpy(), best_items[user_id]}")
    mean_ap += ap

mean_ap /= len(test_users)
mean_ap

100%|██████████| 1000/1000 [00:05<00:00, 181.13it/s]


0.0001666666666666667